## Setup

In [16]:
import tensorflow as tf
tf.enable_eager_execution()
import numpy as np

## Pipe the dataset to a model

In [17]:
X_train = np.load("X_train.npy")
y_train = np.load("y_train.npy")
X_train_preprocessed = X_train / 128
X_train_preprocessed -= 1
print("X_train_preprocessed:", X_train_preprocessed)
print("X_train_preprocessed.shape:", X_train_preprocessed.shape)
print("y_train:", y_train)
print("y_train.shape:", y_train.shape)

X_train_preprocessed: [[[[ 0.2421875  0.2109375  0.21875  ]
   [ 0.2421875  0.2109375  0.21875  ]
   [ 0.2421875  0.2109375  0.21875  ]
   ...
   [ 0.2578125  0.1875     0.2109375]
   [ 0.234375   0.203125   0.1953125]
   [ 0.21875    0.2109375  0.1953125]]

  [[ 0.2421875  0.2109375  0.21875  ]
   [ 0.2421875  0.2109375  0.21875  ]
   [ 0.2421875  0.2109375  0.21875  ]
   ...
   [ 0.2578125  0.1875     0.2109375]
   [ 0.234375   0.203125   0.1953125]
   [ 0.21875    0.2109375  0.1953125]]

  [[ 0.2421875  0.2109375  0.21875  ]
   [ 0.2421875  0.2109375  0.21875  ]
   [ 0.25       0.203125   0.21875  ]
   ...
   [ 0.2578125  0.1875     0.1953125]
   [ 0.234375   0.203125   0.1953125]
   [ 0.21875    0.2109375  0.1953125]]

  ...

  [[-0.625     -0.6171875 -0.5859375]
   [-0.65625   -0.6484375 -0.6171875]
   [-0.6640625 -0.65625   -0.6171875]
   ...
   [ 0.4609375  0.4140625  0.4296875]
   [ 0.4765625  0.40625    0.4296875]
   [ 0.46875    0.3984375  0.40625  ]]

  [[-0.625     -0.63281

In [18]:
label_names = ['undamaged', 'damaged']

In [19]:
mobile_net = tf.keras.applications.MobileNetV2(input_shape=(224, 224, 3), include_top=False)
mobile_net.trainable = False

In [ ]:
model = tf.keras.Sequential([
  mobile_net,
  tf.keras.layers.GlobalAveragePooling2D(),
  tf.keras.layers.Dense(units=len(label_names), activation="softmax")])

In [ ]:
model.compile(optimizer=tf.train.AdamOptimizer(learning_rate=0.01), 
              loss=tf.keras.losses.sparse_categorical_crossentropy,
              metrics=["accuracy"])

In [ ]:
len(model.trainable_variables)

2

In [ ]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenetv2_1.00_224 (Model) (None, 7, 7, 1280)        2257984   
_________________________________________________________________
global_average_pooling2d_2 ( (None, 1280)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 2562      
Total params: 2,260,546
Trainable params: 2,562
Non-trainable params: 2,257,984
_________________________________________________________________


In [ ]:
model.fit(x=X_train_preprocessed, y=y_train, batch_size=100, epochs=100, validation_split=0.1)

Epoch 1/100
20/54 [==========>...................] - ETA: 10:03 - loss: 0.8117 - acc: 0.6535